In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('preprocessed_lyrics.csv')
#df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(333, random_state=42)) #


In [2]:
df

,SName,Lyric,Genre,Artist,lyric_length_processed
0,Run This Town (feat. Rihanna & Kanye West),Rihanna feel come hear scream I addicted it da...,Hip Hop,Jay-Z,334
1,This Is What I do (feat. Waka Flocka & Oj da J...,dirty Birds ay yeah it Guc Big Gucci ay yeah F...,Hip Hop,Gucci Mane,240
2,Knocc Em Down,get ready I fade em run I fade em I need I fad...,Hip Hop,Snoop Dogg,297
3,Dumpin',Eminem Ayo yo motherfucken Eminem dirty Dozen ...,Hip Hop,D12,438
4,Hottest Thing,I look turn brother Song well holla call fight...,Hip Hop,Usher,76
...,...,...,...,...,...
44383,Lonesome Swan,tell wrong feel move feel kid cruel know fair ...,Rock,Glasvegas,24
44384,Treat Me Like You Wanta,you treat like wanta use like well treat like ...,Rock,Johnny Winter,79
44385,Sandcastles,dream dream castle for lonely dream memory dre...,Rock,Journey,11
44386,Tonight,try understand plan hand force alive watch gro...,Rock,Staind,16


In [3]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

Using TensorFlow backend.


In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4722637562428550228
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8959344422092040826
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5807379854736013227
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14949928141
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17383995348540670512
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
]


In [5]:

df["label"] = LabelEncoder().fit_transform(df["Genre"])


In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [7]:
#!pip install tensorflow_hub
import tensorflow_hub as hub


embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [8]:
import tensorflow as tf
import pandas as pd
import keras

from sklearn import preprocessing
from keras.utils import np_utils
import numpy as np


y = list(df['label'])
x = list(df['Lyric'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(le, one_hot):
    print(one_hot)
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


x_enc = x
y_enc = encode(le, y)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.1, random_state=42)

In [10]:
x_train.shape


(39949,)

In [11]:
from keras.layers import Input, Lambda, Dense
import tensorflow as tf

input_text = Input(shape=(1,), dtype=tf.string)
print(input_text)

Tensor("input_1:0", shape=(?, 1), dtype=string)


In [12]:
from tensorflow.keras.layers import Input, Lambda, Bidirectional, Dense, Dropout,LSTM
from tensorflow.keras.models import Model
import keras.backend as K

def ELMoEmbedding(input_text):
    return embed(tf.reshape(tf.cast(input_text, tf.string), [-1]), signature="default", as_dict=True)["elmo"]

input_layer = Input(shape=(1,), dtype="string", name="Input_layer")
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ), name="Elmo_Embedding")(input_layer)
BiLSTM = Bidirectional(LSTM(128, return_sequences= False, recurrent_dropout=0.3, dropout=0.3), name="BiLSTM")(embedding_layer)
Dense_layer_1 = Dense(32, activation='relu')(BiLSTM)
pred = Dense(3, activation='sigmoid')(Dense_layer_1)
model = Model(inputs=[input_layer], outputs=pred)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])


with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer()) 
    session.run(tf.tables_initializer())
    model_elmo = model.fit(x_train, y_train, epochs=1, batch_size=16)
    train_prediction = model.predict(x_test)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[2022-01-08 11:23:01.976 ip-172-31-30-92:10014 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-01-08 11:23:01.997 ip-172-31-30-92:10014 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Train on 39949 samples
14080/39949 [=========>....................] - ETA: 43:28 - loss: 0.5034 - acc: 0.7379

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[32,1024,1450,44] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node Elmo_Embedding/module_apply_default/bilm/CNN/Conv2D_6}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/mul/_265]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[32,1024,1450,44] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node Elmo_Embedding/module_apply_default/bilm/CNN/Conv2D_6}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
predicts = train_prediction

In [ ]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [ ]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

from sklearn.metrics import accuracy_score

print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

Large diff between acc on training vs testing (might be overfit)?